# Exploring Hacker News Posts

Hacker News is a site started by the startup incubator [Y Combinator](https://www.ycombinator.com/), where user-submitted stories (known as "posts") receive votes and comments, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of the Hacker News listings can get hundreds of thousands of visitors as a result.

The original dataset can be found on [Kaggle](https://www.kaggle.com/hacker-news/hacker-news-posts), but in this analysis we only look at a dataset comprising a [subsample](https://dq-content.s3.amazonaws.com/356/hacker_news.csv) of the original dataset. The original has ca. 300,000  cases, but the subsample only comprises ca. 20,000 rows. In this sample, submissions that didn't receive any comments were deleted and then from the remaining posts 20,000 were drawn randomly.

Here's an overview of the columns in the dataset:

|Index | Name | Description |
|:---:|:---|:---|
| 0  | id | Identifier  |
| 1  | title | title of the post (self explanatory) |
| 2   | url | the url of the item being linked to |
| 3  | num_points | the number of upvotes the post received |
| 4  | num_comments | the number of comments the post received |
| 5 | author | the name of the account that made the post |
| 6  | created_at | the date and time the post was made (the time zone is US Eastern Time) |


## Data exploration

We remove the header and place it separately from the rest. We explore the first few rows of data using the `explore_data()`function.

In [ ]:
from csv import reader

### The Hacker News data set ###
opened_file = open('C:\\Users\\timmu\\Documents\\Dataquest\\hacker_news.csv', encoding='utf8')
read_file = reader(opened_file)
hn = list(read_file)
hn_header = hn[0]
hn = hn[1:]

##Function to explore the data
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

##Column names for hn##
print(hn_header)
print('\n')
explore_data(hn, 0, 5, True)

## Extracting Ask HN and Show HN Posts

We're now ready to filter our data. Since we're only concerned with post titles beginning with `Ask HN` or `Show HN`, we'll create new lists of lists containing just the data for those titles. We use the `startswith` method on the post titles and then separate the posts into two different lists based on whether the title string starts with the phrase that we're after. 

In [2]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    if title.lower().startswith('ask hn'): # we convert all titles to lowercase
        ask_posts.append(row)
    elif title.lower().startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

1744
1162
17194


In [3]:
explore_data(ask_posts, 0, 5)
print('\n')
explore_data(show_posts, 0, 5)

['12296411', 'Ask HN: How to improve my personal website?', '', '2', '6', 'ahmedbaracat', '8/16/2016 9:55']


['10610020', 'Ask HN: Am I the only one outraged by Twitter shutting down share counts?', '', '28', '29', 'tkfx', '11/22/2015 13:43']


['11610310', 'Ask HN: Aby recent changes to CSS that broke mobile?', '', '1', '1', 'polskibus', '5/2/2016 10:14']


['12210105', 'Ask HN: Looking for Employee #3 How do I do it?', '', '1', '3', 'sph130', '8/2/2016 14:20']


['10394168', 'Ask HN: Someone offered to buy my browser extension from me. What now?', '', '28', '17', 'roykolak', '10/15/2015 16:38']




['10627194', 'Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform', 'https://iot.seeed.cc', '26', '22', 'kfihihc', '11/25/2015 14:03']


['10646440', 'Show HN: Something pointless I made', 'http://dn.ht/picklecat/', '747', '102', 'dhotson', '11/29/2015 22:46']


['11590768', 'Show HN: Shanhu.io, a programming playground powered by e8vm', 'https://shanhu.io', '1', 

We have 1744 posts in the 'ask' category, 1162 in the 'show' category and 17,194 in the 'other' category. 

Next, let's determine if ask posts or show posts receive more comments on average.

## Calculating the Average Number of Comments for Ask HN and Show HN Posts


In [4]:
# couting comments in ask_posts
total_ask_comments = 0
for row in ask_posts:
    num_comments = int(row[4])
    total_ask_comments = total_ask_comments + num_comments
avg_ask_comments = total_ask_comments / len(ask_posts)
print("Ask posts: ", round(avg_ask_comments, 2))

# couting comments in show_posts
total_show_comments = 0
for row in show_posts:
    num_comments = int(row[4])
    total_show_comments = total_show_comments + num_comments
avg_show_comments = total_show_comments / len(show_posts)
print("Show posts: ", round(avg_show_comments, 2))

# couting comments in other_posts
total_other_comments = 0
for row in other_posts:
    num_comments = int(row[4])
    total_other_comments = total_other_comments + num_comments
avg_other_comments = total_other_comments / len(other_posts)
print("Other posts: ", round(avg_other_comments, 2))

Ask posts:  14.04
Show posts:  10.32
Other posts:  26.87


Looking at the average number of comments for the 'ask' and 'show' categories, we find that 'ask' posts on average receive ca. 4 more comments (14.04) than 'show' posts (10.32). People seem like to be helpful and more willing to comment when asked for their help or opinion. They seem to make fewer (unsolicited) comments on posts where people show what they did. But looking at the average number of comments in all other categories, we see that people are still more responsive there (26.87 comments per post). This might relate to the fact that other posts comprise general discussion topics on broader themes, leading to a high average amount.

Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts. 

## Finding the Number of Ask Posts and Comments by Hour Created

Next, we'll determine if ask posts created at a certain time are more likely to attract comments. We'll use the following steps to perform this analysis:

1. Calculate the number of ask posts created in each hour of the day, along with the number of        comments received.
2. Calculate the average number of comments ask posts receive by hour created.

We will import the `datetime` module for this and work with the data in the `created_at` column.

In [5]:
import datetime as dt

In [6]:
# appending a list with two elements to the result list
result_list = []
for row in ask_posts:
    created_at = row[6]
    num_comments = int(row[4])
    result_list.append([created_at, num_comments])
   
print(result_list[0:5])

[['8/16/2016 9:55', 6], ['11/22/2015 13:43', 29], ['5/2/2016 10:14', 1], ['8/2/2016 14:20', 3], ['10/15/2015 16:38', 17]]


In [7]:
counts_by_hour = {}
comments_by_hour = {} 
for row in result_list:
    date_dt =  dt.datetime.strptime(row[0], "%m/%d/%Y %H:%M") # extracting the date
    hour_dt = date_dt.hour # extracting the hour from the datetime object
    hour_str = str(hour_dt) # converting the hour element into a string
    num_comments = row[1] # assigning the number of comments to a variable
    
    # frequency list for the number of posts by hour and the number of comments by hour
    if hour_str not in counts_by_hour: 
        counts_by_hour[hour_str] = 1
        comments_by_hour[hour_str] = num_comments
    else:
        counts_by_hour[hour_str] += 1
        comments_by_hour[hour_str] += num_comments        

In [8]:
print(counts_by_hour)

{'9': 45, '13': 85, '10': 59, '14': 107, '16': 108, '23': 68, '12': 73, '17': 100, '15': 116, '21': 109, '20': 80, '2': 58, '18': 109, '3': 54, '5': 46, '19': 110, '1': 60, '22': 71, '8': 48, '4': 47, '0': 55, '6': 44, '7': 34, '11': 58}


In [10]:
print(comments_by_hour)

{'9': 251, '13': 1253, '10': 793, '14': 1416, '16': 1814, '23': 543, '12': 687, '17': 1146, '15': 4477, '21': 1745, '20': 1722, '2': 1381, '18': 1439, '3': 421, '5': 464, '19': 1188, '1': 683, '22': 479, '8': 492, '4': 337, '0': 447, '6': 397, '7': 267, '11': 641}


This is all nice and well, but we want to have the average number of comments by hour. So we calculate this by looping through the keys of `comments_by_hour` and appending the averages to the new dictionary `avg_by_hour`.

In [12]:
avg_by_hour = []
for hour in comments_by_hour:
    avg_by_hour.append([hour, comments_by_hour[hour] / counts_by_hour[hour]])

print(avg_by_hour)

[['9', 5.5777777777777775], ['13', 14.741176470588234], ['10', 13.440677966101696], ['14', 13.233644859813085], ['16', 16.796296296296298], ['23', 7.985294117647059], ['12', 9.41095890410959], ['17', 11.46], ['15', 38.5948275862069], ['21', 16.009174311926607], ['20', 21.525], ['2', 23.810344827586206], ['18', 13.20183486238532], ['3', 7.796296296296297], ['5', 10.08695652173913], ['19', 10.8], ['1', 11.383333333333333], ['22', 6.746478873239437], ['8', 10.25], ['4', 7.170212765957447], ['0', 8.127272727272727], ['6', 9.022727272727273], ['7', 7.852941176470588], ['11', 11.051724137931034]]


We still need to sort this dictionary in order ot make sense of the results. Therefore we first swap the columns of `avg_by_hour`.

In [14]:
swap_avg_by_hour = []
for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])

print(swap_avg_by_hour)

[[5.5777777777777775, '9'], [14.741176470588234, '13'], [13.440677966101696, '10'], [13.233644859813085, '14'], [16.796296296296298, '16'], [7.985294117647059, '23'], [9.41095890410959, '12'], [11.46, '17'], [38.5948275862069, '15'], [16.009174311926607, '21'], [21.525, '20'], [23.810344827586206, '2'], [13.20183486238532, '18'], [7.796296296296297, '3'], [10.08695652173913, '5'], [10.8, '19'], [11.383333333333333, '1'], [6.746478873239437, '22'], [10.25, '8'], [7.170212765957447, '4'], [8.127272727272727, '0'], [9.022727272727273, '6'], [7.852941176470588, '7'], [11.051724137931034, '11']]


Now we can sort this list, since the values are the first element.

In [17]:
sorted_swap = sorted(swap_avg_by_hour, reverse=True)
print(sorted_swap)

[[38.5948275862069, '15'], [23.810344827586206, '2'], [21.525, '20'], [16.796296296296298, '16'], [16.009174311926607, '21'], [14.741176470588234, '13'], [13.440677966101696, '10'], [13.233644859813085, '14'], [13.20183486238532, '18'], [11.46, '17'], [11.383333333333333, '1'], [11.051724137931034, '11'], [10.8, '19'], [10.25, '8'], [10.08695652173913, '5'], [9.41095890410959, '12'], [9.022727272727273, '6'], [8.127272727272727, '0'], [7.985294117647059, '23'], [7.852941176470588, '7'], [7.796296296296297, '3'], [7.170212765957447, '4'], [6.746478873239437, '22'], [5.5777777777777775, '9']]


In [24]:
print("Top 5 Hours for Ask Posts Comments")
# Loop through the first five lists
for row in sorted_swap[0:5]:
    hour = row[1]
    avg = row[0]
    hour_dt = dt.datetime.strptime(hour, "%H")
    format_hour = hour_dt.strftime("%H:%M")
    print("{}: {:.2f} average comments per post".format(format_hour, avg))

Top 5 Hours for Ask Posts Comments
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


Since these times refer to Eastern Standard Time in the US, we should add 6 hours (during summer) in order to get CET. Therefore, in order to receive the most comments based on our time zone 21:00, 8:00, 02:00, 22:00 and 3:00 would work well.